<h1>521H0287 - Văn Công Nguyên Phong</h1>
<h1>521H0285 - Phạm Trần Tiến Phát</h1>
<h1>521H0302 - Trương Công Thành</h1>

**Feed Forward Neural Network**

A feed forward neural network (FFNN) is one of the two broad types of artificial neural network, characterized by direction of the flow of information between its layers. Its flow is uni-directional, meaning that the information in the model flows in only one direction—forward—from the input nodes, through the hidden nodes (if any) and to the output nodes, without any cycles or loops, in contrast to recurrent neural networks, which have a bi-directional flow. Modern feedforward networks are trained using the backpropagation method and are colloquially referred to as the "vanilla" neural networks.



In [222]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from scipy.stats import zscore, boxcox
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, BatchNormalization, Activation, Input
import warnings

warnings.simplefilter("ignore")
pd.set_option('display.max_columns', None)

In [223]:
current_directory = os.getcwd()
csv_filename = 'data.csv'
csv_path = os.path.join(current_directory, "data" ,csv_filename)

In [224]:
billionaires = pd.read_csv(csv_path)
billionaires.head(5)  

,rank,finalWorth,category,personName,age,country,city,source,industries,countryOfCitizenship,organization,selfMade,status,gender,birthDate,lastName,firstName,title,date,state,residenceStateRegion,birthYear,birthMonth,birthDay,cpi_country,cpi_change_country,gdp_country,gross_tertiary_education_enrollment,gross_primary_education_enrollment_country,life_expectancy_country,tax_revenue_country_country,total_tax_rate_country,population_country,latitude_country,longitude_country
0,1,211000,Fashion & Retail,Bernard Arnault & family,74.0,France,Paris,LVMH,Fashion & Retail,France,LVMH Moët Hennessy Louis Vuitton,False,U,M,3/5/1949 0:00,Arnault,Bernard,Chairman and CEO,4/4/2023 5:01,NaN,NaN,1949.0,3.0,5.0,110.05,1.1,"$2,715,518,274,227",65.6,102.5,82.5,24.2,60.7,67059887.0,46.227638,2.213749
1,2,180000,Automotive,Elon Musk,51.0,United States,Austin,"Tesla, SpaceX",Automotive,United States,Tesla,True,D,M,6/28/1971 0:00,Musk,Elon,CEO,4/4/2023 5:01,Texas,South,1971.0,6.0,28.0,117.24,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,328239523.0,37.090240,-95.712891
2,3,114000,Technology,Jeff Bezos,59.0,United States,Medina,Amazon,Technology,United States,Amazon,True,D,M,1/12/1964 0:00,Bezos,Jeff,Chairman and Founder,4/4/2023 5:01,Washington,West,1964.0,1.0,12.0,117.24,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,328239523.0,37.090240,-95.712891
3,4,107000,Technology,Larry Ellison,78.0,United States,Lanai,Oracle,Technology,United States,Oracle,True,U,M,8/17/1944 0:00,Ellison,Larry,CTO and Founder,4/4/2023 5:01,Hawaii,West,1944.0,8.0,17.0,117.24,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,328239523.0,37.090240,-95.712891
4,5,106000,Finance & Investments,Warren Buffett,92.0,United States,Omaha,Berkshire Hathaway,Finance & Investments,United States,Berkshire Hathaway Inc. (Cl A),True,D,M,8/30/1930 0:00,Buffett,Warren,CEO,4/4/2023 5:01,Nebraska,Midwest,1930.0,8.0,30.0,117.24,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,328239523.0,37.090240,-95.712891


In [225]:
billionaires.describe()

,rank,finalWorth,age,birthYear,birthMonth,birthDay,cpi_country,cpi_change_country,gross_tertiary_education_enrollment,gross_primary_education_enrollment_country,life_expectancy_country,tax_revenue_country_country,total_tax_rate_country,population_country,latitude_country,longitude_country
count,2640.000000,2640.000000,2575.000000,2564.000000,2564.000000,2564.000000,2456.000000,2456.000000,2458.000000,2459.000000,2458.000000,2457.000000,2458.000000,2.476000e+03,2476.000000,2476.000000
mean,1289.159091,4623.787879,65.140194,1957.183307,5.740250,12.099844,127.755204,4.364169,67.225671,102.858520,78.122823,12.546235,43.963344,5.102053e+08,34.903592,12.583156
std,739.693726,9834.240939,13.258098,13.282516,3.710085,9.918876,26.452951,3.623763,21.343426,4.710977,3.730099,5.368625,12.145296,5.542447e+08,17.003497,86.762989
min,1.000000,1000.000000,18.000000,1921.000000,1.000000,1.000000,99.550000,-1.900000,4.000000,84.700000,54.300000,0.100000,9.900000,3.801900e+04,-40.900557,-106.346771
25%,659.000000,1500.000000,56.000000,1948.000000,2.000000,1.000000,117.240000,1.700000,50.600000,100.200000,77.000000,9.600000,36.600000,6.683440e+07,35.861660,-95.712891
50%,1312.000000,2300.000000,65.000000,1957.000000,6.000000,11.000000,117.240000,2.900000,65.600000,101.800000,78.500000,9.600000,41.200000,3.282395e+08,37.090240,10.451526
75%,1905.000000,4200.000000,75.000000,1966.000000,9.000000,21.000000,125.080000,7.500000,88.200000,102.600000,80.900000,12.800000,59.100000,1.366418e+09,40.463667,104.195397
max,2540.000000,211000.000000,101.000000,2004.000000,12.000000,31.000000,288.570000,53.500000,136.600000,142.100000,84.200000,37.200000,106.300000,1.397715e+09,61.924110,174.885971


In [226]:
billionaires['selfMade'] = billionaires['selfMade'].astype("object")
billionaires.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2640 entries, 0 to 2639
Data columns (total 35 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   rank                                        2640 non-null   int64  
 1   finalWorth                                  2640 non-null   int64  
 2   category                                    2640 non-null   object 
 3   personName                                  2640 non-null   object 
 4   age                                         2575 non-null   float64
 5   country                                     2602 non-null   object 
 6   city                                        2568 non-null   object 
 7   source                                      2640 non-null   object 
 8   industries                                  2640 non-null   object 
 9   countryOfCitizenship                        2640 non-null   object 
 10  organization

In [227]:
## Getting total of missing values
billionaires.isna().sum().sort_values(ascending = False)

organization                                  2315
title                                         2301
residenceStateRegion                          1893
state                                         1887
cpi_change_country                             184
cpi_country                                    184
tax_revenue_country_country                    183
total_tax_rate_country                         182
life_expectancy_country                        182
gross_tertiary_education_enrollment            182
gross_primary_education_enrollment_country     181
latitude_country                               164
population_country                             164
gdp_country                                    164
longitude_country                              164
birthDate                                       76
birthYear                                       76
birthMonth                                      76
birthDay                                        76
city                           

In [228]:
billionaires.columns
# Specifying input and target variables
columns_to_drop = ['personName', 'lastName', 'firstName', 'birthYear', 'birthMonth',
                   'birthDay', 'birthDate', 'date', 'organization', 'residenceStateRegion', 'state',
                   'title']

billionaires.drop(columns=columns_to_drop, axis=1, inplace=True)

cat = billionaires.select_dtypes(include="object").columns
num = billionaires.select_dtypes(include=['int', 'float']).columns

In [229]:
print(cat)

Index(['category', 'country', 'city', 'source', 'industries',
       'countryOfCitizenship', 'selfMade', 'status', 'gender', 'gdp_country'],
      dtype='object')


In [230]:
for i in billionaires.columns:
    if i in cat:
        billionaires[i] = billionaires[i].fillna(billionaires[i].mode()[0])
    else:
        avg = billionaires[i].mean()
        md = billionaires[i].median()
        mod = billionaires[i].mode()

        # Predict missing values        
        imputer = KNNImputer(n_neighbors=3)
        knr = imputer.fit_transform(billionaires[num])

        # Choice one values from randomly
        rn_index = np.random.choice(3)
        bf = billionaires[i].shift(-1).iloc[rn_index]
        af = billionaires[i].shift(+1).iloc[rn_index]

        n = [avg, md, mod.values[0], bf, af, knr[rn_index][rn_index]]

        rn_value = n[rn_index]
        rn_value

        # Fill data by selected value
        billionaires[i] = billionaires[i].fillna(rn_value)

In [231]:
billionaires.isna().sum()/len(billionaires)*100

rank                                          0.0
finalWorth                                    0.0
category                                      0.0
age                                           0.0
country                                       0.0
city                                          0.0
source                                        0.0
industries                                    0.0
countryOfCitizenship                          0.0
selfMade                                      0.0
status                                        0.0
gender                                        0.0
cpi_country                                   0.0
cpi_change_country                            0.0
gdp_country                                   0.0
gross_tertiary_education_enrollment           0.0
gross_primary_education_enrollment_country    0.0
life_expectancy_country                       0.0
tax_revenue_country_country                   0.0
total_tax_rate_country                        0.0


In [232]:
billionaires[num].skew()

rank                                          -0.006357
finalWorth                                    10.012367
age                                           -0.079037
cpi_country                                    2.707725
cpi_change_country                             4.465587
gross_tertiary_education_enrollment           -0.350761
gross_primary_education_enrollment_country     2.401476
life_expectancy_country                       -1.074991
tax_revenue_country_country                    1.673160
total_tax_rate_country                         0.265202
population_country                             0.944884
latitude_country                              -1.810701
longitude_country                             -0.146089
dtype: float64

In [233]:
billionaires['finalWorth'] = boxcox(billionaires['finalWorth'])[0]

In [234]:
dict_le = {}

for i in cat:
    dict_le[i] = LabelEncoder()
    billionaires[i] = dict_le[i].fit_transform(billionaires[i])

In [235]:
scaler = StandardScaler()

X = billionaires.drop('selfMade',axis=1)
x = scaler.fit_transform(X)
y = billionaires['selfMade']

In [236]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3, random_state=45)

In [237]:
ffnn_normal = Sequential([
    Dense(128, activation="relu", input_shape=(xtrain.shape[1],)),
    Dense(64, activation="relu"),
    Dense(32, activation="relu"),
    Dense(16, activation="relu"),
    Dense(8, activation="relu"),
    Dense(1, activation="sigmoid")
])

In [238]:
ffnn_normal.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
ffnn_normal.fit(xtrain, ytrain, epochs=10, batch_size=32, validation_data=(xtest, ytest))

Epoch 1/10
58/58 [==============================] - 1s 5ms/step - loss: 0.5373 - accuracy: 0.6883 - val_loss: 0.4918 - val_accuracy: 0.7033
Epoch 2/10
58/58 [==============================] - 0s 3ms/step - loss: 0.4834 - accuracy: 0.7543 - val_loss: 0.4829 - val_accuracy: 0.7664
Epoch 3/10
58/58 [==============================] - 0s 3ms/step - loss: 0.4681 - accuracy: 0.7749 - val_loss: 0.4866 - val_accuracy: 0.7816
Epoch 4/10
58/58 [==============================] - 0s 3ms/step - loss: 0.4523 - accuracy: 0.7906 - val_loss: 0.4907 - val_accuracy: 0.7702
Epoch 5/10
58/58 [==============================] - 0s 3ms/step - loss: 0.4435 - accuracy: 0.7998 - val_loss: 0.4850 - val_accuracy: 0.7904
Epoch 6/10
58/58 [==============================] - 0s 3ms/step - loss: 0.4318 - accuracy: 0.8047 - val_loss: 0.4929 - val_accuracy: 0.7790
Epoch 7/10
58/58 [==============================] - 0s 3ms/step - loss: 0.4297 - accuracy: 0.8122 - val_loss: 0.4945 - val_accuracy: 0.7841
Epoch 8/10
58/58 [==

In [239]:
y_pred = ffnn_normal.predict(xtest)
y_pred_classes = np.round(y_pred)

cm = confusion_matrix(ytest, y_pred_classes)
print("Confusion Matrix:")
print(cm)

report = classification_report(ytest, y_pred_classes)
print("Classification Report:")
print(report)

25/25 [==============================] - 0s 1ms/step
Confusion Matrix:
[[117 134]
 [ 51 490]]
Classification Report:
              precision    recall  f1-score   support

           0       0.70      0.47      0.56       251
           1       0.79      0.91      0.84       541

    accuracy                           0.77       792
   macro avg       0.74      0.69      0.70       792
weighted avg       0.76      0.77      0.75       792



In [240]:
model_ffnn = Sequential([
    Dense(128, activation="relu", input_shape=(xtrain.shape[1],), kernel_regularizer=keras.regularizers.l2(0.01)),
    # Dense(128, activation="relu", kernel_regularizer=keras.regularizers.l2(0.01)),
    Dense(64, activation="relu", kernel_regularizer=keras.regularizers.l2(0.01)),
    Dropout(0.5),
    Dense(32, activation="relu"),
    Dropout(0.5),
    Dense(16, activation="relu", kernel_regularizer=keras.regularizers.l2(0.01)),
    Dense(8, activation="relu"),
    Dense(1, activation="sigmoid",kernel_regularizer=keras.regularizers.l2(0.01))
])

In [241]:
model_ffnn.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

#Early Stopping
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

model_ffnn.fit(xtrain, ytrain, epochs=30, batch_size=32, validation_data=(xtest, ytest) ,callbacks=[early_stopping])

Epoch 1/30
58/58 [==============================] - 1s 6ms/step - loss: 1.7931 - accuracy: 0.5931 - val_loss: 1.4296 - val_accuracy: 0.6831
Epoch 2/30
58/58 [==============================] - 0s 3ms/step - loss: 1.2534 - accuracy: 0.6970 - val_loss: 1.0487 - val_accuracy: 0.7134
Epoch 3/30
58/58 [==============================] - 0s 3ms/step - loss: 0.9652 - accuracy: 0.7116 - val_loss: 0.8425 - val_accuracy: 0.7715
Epoch 4/30
58/58 [==============================] - 0s 4ms/step - loss: 0.7959 - accuracy: 0.7376 - val_loss: 0.7184 - val_accuracy: 0.7790
Epoch 5/30
58/58 [==============================] - 0s 3ms/step - loss: 0.7033 - accuracy: 0.7538 - val_loss: 0.6472 - val_accuracy: 0.7778
Epoch 6/30
58/58 [==============================] - 0s 3ms/step - loss: 0.6495 - accuracy: 0.7516 - val_loss: 0.6084 - val_accuracy: 0.7790
Epoch 7/30
58/58 [==============================] - 0s 4ms/step - loss: 0.6035 - accuracy: 0.7565 - val_loss: 0.5765 - val_accuracy: 0.7803
Epoch 8/30
58/58 [==

In [242]:
y_pred = model_ffnn.predict(xtest)
y_pred_classes = np.round(y_pred)

cm = confusion_matrix(ytest, y_pred_classes)
print("Confusion Matrix:")
print(cm)

report = classification_report(ytest, y_pred_classes)
print("Classification Report:")
print(report)

25/25 [==============================] - 0s 2ms/step
Confusion Matrix:
[[126 125]
 [ 60 481]]
Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.50      0.58       251
           1       0.79      0.89      0.84       541

    accuracy                           0.77       792
   macro avg       0.74      0.70      0.71       792
weighted avg       0.76      0.77      0.76       792



**Recurrent Neural Network**

Recurrent neural network (RNN) is a class of artificial neural networks where connections between nodes form a directed graph along a time sequence. This allows the network to take temporary action.

In [243]:
rnn_normal = Sequential([
    LSTM(units=50, input_shape=(22,1)),
    Dense(units=64, activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=16, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [244]:
rnn_normal.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
rnn_normal.fit( xtrain, ytrain, epochs=10, batch_size=32, validation_data=(xtest, ytest))

Epoch 1/10
58/58 [==============================] - 3s 22ms/step - loss: 0.6317 - accuracy: 0.6775 - val_loss: 0.5737 - val_accuracy: 0.6831
Epoch 2/10
58/58 [==============================] - 1s 14ms/step - loss: 0.5528 - accuracy: 0.6916 - val_loss: 0.5200 - val_accuracy: 0.7184
Epoch 3/10
58/58 [==============================] - 1s 13ms/step - loss: 0.5263 - accuracy: 0.7067 - val_loss: 0.5081 - val_accuracy: 0.7222
Epoch 4/10
58/58 [==============================] - 1s 12ms/step - loss: 0.5202 - accuracy: 0.7127 - val_loss: 0.5013 - val_accuracy: 0.7285
Epoch 5/10
58/58 [==============================] - 1s 11ms/step - loss: 0.5158 - accuracy: 0.7224 - val_loss: 0.4988 - val_accuracy: 0.7323
Epoch 6/10
58/58 [==============================] - 1s 11ms/step - loss: 0.5136 - accuracy: 0.7256 - val_loss: 0.5004 - val_accuracy: 0.7412
Epoch 7/10
58/58 [==============================] - 1s 11ms/step - loss: 0.5138 - accuracy: 0.7224 - val_loss: 0.4950 - val_accuracy: 0.7437
Epoch 8/10
58

In [245]:
y_pred = rnn_normal.predict(xtest)
y_pred_classes = np.round(y_pred)

cm = confusion_matrix(ytest, y_pred_classes)
print("Confusion Matrix:")
print(cm)

report = classification_report(ytest, y_pred_classes)
print("Classification Report:")
print(report)

25/25 [==============================] - 0s 4ms/step
Confusion Matrix:
[[102 149]
 [ 58 483]]
Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.41      0.50       251
           1       0.76      0.89      0.82       541

    accuracy                           0.74       792
   macro avg       0.70      0.65      0.66       792
weighted avg       0.72      0.74      0.72       792



In [246]:
#Build the RNN model
model_rnn = Sequential([
    LSTM(units=50, input_shape=(22,1)),
    Dropout(0.2),
    BatchNormalization(),
    Dense(units=64, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(units=32, activation='relu'),
    Dropout(0.2),
    BatchNormalization(),
    Dense(1, activation='sigmoid')
])

In [247]:
#Compile the model
model_rnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_rnn.fit(
    xtrain, ytrain,
    epochs=50, batch_size=16,
    validation_data=(xtest, ytest),
    callbacks=[early_stopping],
)

Epoch 1/50
116/116 [==============================] - 5s 16ms/step - loss: 0.7183 - accuracy: 0.5904 - val_loss: 0.6517 - val_accuracy: 0.6831
Epoch 2/50
116/116 [==============================] - 1s 10ms/step - loss: 0.6330 - accuracy: 0.6596 - val_loss: 0.6606 - val_accuracy: 0.7172
Epoch 3/50
116/116 [==============================] - 1s 11ms/step - loss: 0.6061 - accuracy: 0.6780 - val_loss: 0.6109 - val_accuracy: 0.7071
Epoch 4/50
116/116 [==============================] - 1s 11ms/step - loss: 0.5831 - accuracy: 0.6861 - val_loss: 0.5648 - val_accuracy: 0.7424
Epoch 5/50
116/116 [==============================] - 1s 11ms/step - loss: 0.5628 - accuracy: 0.7002 - val_loss: 0.5272 - val_accuracy: 0.7285
Epoch 6/50
116/116 [==============================] - 1s 10ms/step - loss: 0.5790 - accuracy: 0.6948 - val_loss: 0.5196 - val_accuracy: 0.7374
Epoch 7/50
116/116 [==============================] - 1s 11ms/step - loss: 0.5698 - accuracy: 0.7175 - val_loss: 0.5145 - val_accuracy: 0.7298

In [248]:
y_pred = model_rnn.predict(xtest)
y_pred_classes = np.round(y_pred)

cm = confusion_matrix(ytest, y_pred_classes)
print("Confusion Matrix:")
print(cm)

report = classification_report(ytest, y_pred_classes)
print("Classification Report:")
print(report)

25/25 [==============================] - 0s 4ms/step
Confusion Matrix:
[[114 137]
 [ 54 487]]
Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.45      0.54       251
           1       0.78      0.90      0.84       541

    accuracy                           0.76       792
   macro avg       0.73      0.68      0.69       792
weighted avg       0.75      0.76      0.74       792

